In [2]:
import pandas as pd
import datetime as dt
import swifter

In [3]:
# f = "/home/wmnlab/diag_log_sm05_2022-12-22_15-02-55.txt_rrc.csv"
f = "/home/wmnlab/diag_log_sm08_2022-12-22_15-26-10.txt_rrc.csv"
df = pd.read_csv(f)
df["time"] = df["time"].swifter.apply(lambda x: pd.to_datetime(x) + dt.timedelta(hours=8))

In [18]:
# df.columns

In [19]:
def HO_trajectory(df):
    def NR_OTA():
            if df["type_id"].iloc[i] == "5G_NR_RRC_OTA_Packet":
                return True
            else:
                return False

    def find_1st_after(target, look_after=1):
            for j in range(i, len(df)):
                t_ = df["time"].iloc[j]
                if (t_ - t).total_seconds() > look_after:
                    return None, None
                if df[target].iloc[j] not in [0,'0']:
                    return t_, j

    def find_1st_before(target, look_before=1):
        for j in range(i, -1, -1):
            t_ = df["time"].iloc[j]
            if (t - t_).total_seconds() > look_before:
                return None, None
            if df[target].iloc[j] not in [0,'0']:
                return t_, j


    pci_0, freq_0, t_0 = df["PCI"].iloc[0], df["Freq"].iloc[0], df["time"].iloc[0]
    print(f"{t_0} Initial {pci_0} {freq_0}")
    for i in range(len(df)):
        if NR_OTA():
                continue

        t = df["time"].iloc[i]
        pci =  df["PCI"].iloc[i]
        # print(pci)
        others = ''
        if df["rrcConnectionRelease"].iloc[i] == 1:
            print(f"{t} RRC Release at {pci}.")

        if df["rrcConnectionSetup"].iloc[i] == 1:
            print(f"{t} CONNECTED setup at {pci}")

        if df["lte-rrc.t304"].iloc[i] == 1:
            end, ii = find_1st_after('rrcConnectionReconfigurationComplete')
            serv_cell, target_cell = df["PCI"].iloc[i], df['lte_targetPhysCellId'].iloc[i]
            serv_freq, target_freq = df["Freq"].iloc[i], df['Freq'].iloc[ii]
            if df["SCellToAddMod-r10"].iloc[i] == 1:
                n =len(str(df["SCellIndex-r10.1"].iloc[i]).split('@'))
                others=f'Set up {n} SCell.'
            else:
                others=None
            
            if serv_freq != target_freq:
                others += " Inter freq. HO"
            if df["nr-rrc.t304"].iloc[i] == 1 and df["dualConnectivityPHR: setup (1)"].iloc[i] == 1:
                if serv_cell == target_cell and serv_freq == target_freq:
                    nr_target_cell = df["nr_physCellId"].iloc[i]
                    print(f"{t} eNB_to_ENDC(NR setup) to {nr_target_cell} | {others}")
                else:
                    print(f"{t} MN_HO {serv_cell} -> {target_cell} | {others}")
            else:
                if serv_cell == target_cell and serv_freq == target_freq:
                    a, b = find_1st_before("scgFailureInformationNR-r15")
                    if a is not None:
                        others += " Caused by scg-failure."
                    print(f"{t} gNB_Rel | {others}")
                else:
                    print(f"{t} LTE_HO {serv_cell} -> {target_cell} | {others}")
        
        if df["nr-rrc.t304"].iloc[i] == 1 and not df["dualConnectivityPHR: setup (1)"].iloc[i] == 1:
            origin_cell = df["PCI"].iloc[i+1]
            target_cell = df["nr_physCellId"].iloc[i]
            print(f"{t} gNB_HO {origin_cell} -> {target_cell}")

        if df["rrcConnectionReestablishmentRequest"].iloc[i] == 1:
            end, _ = find_1st_after('rrcConnectionReestablishmentComplete', look_after=1)
            b, _ = find_1st_after('rrcConnectionReestablishmentReject', look_after=1)
            others = df["reestablishmentCause"].iloc[i]
            if end is not None: 
                # Type II
                target_cell = df['physCellId.3'].iloc[i]
                print(f"{t} RLF II Failed at {target_cell}, recconected to {pci} | {others}")
            else: 
                # Type III
                print(f"{t} RLF III {others}")
            
        if df["scgFailureInformationNR-r15"].iloc[i] == 1:
            others = df["failureType-r15"].iloc[i]
            print(f"{t} SCG RLF | {others}")

In [20]:
HO_trajectory(df)

2022-12-22 15:25:59.047565 Initial 362 1750
2022-12-22 15:26:02.088951 gNB_HO 362 -> 350
2022-12-22 15:26:35.576641 CONNECTED setup at 362
2022-12-22 15:26:35.789162 eNB_to_ENDC(NR setup) to 350 | Set up 1 SCell.
2022-12-22 15:26:54.379002 RRC Release at 362.
2022-12-22 15:27:08.486496 CONNECTED setup at 362
2022-12-22 15:27:08.703003 eNB_to_ENDC(NR setup) to 350 | Set up 1 SCell.
2022-12-22 15:29:24.414549 gNB_HO 350 -> 362
2022-12-22 15:30:08.291335 MN_HO 362 -> 73 | Set up 1 SCell.
2022-12-22 15:30:09.054408 gNB_HO 362 -> 370
2022-12-22 15:30:10.607753 MN_HO 73 -> 378 | Set up 1 SCell.
2022-12-22 15:30:16.215805 RLF II Failed at 378, recconected to 73 | otherFailure (2)
2022-12-22 15:30:16.952106 LTE_HO 73 -> 73 | Set up 1 SCell. Inter freq. HO
2022-12-22 15:30:17.104613 eNB_to_ENDC(NR setup) to 73 | Set up 1 SCell.
2022-12-22 15:30:50.754038 SCG RLF | rlc-MaxNumRetx (2)
2022-12-22 15:30:50.790831 gNB_Rel | Set up 1 SCell. Caused by scg-failure.
2022-12-22 15:31:07.281265 LTE_HO 73 